## Наиболее точно порекомендовать список из 10 книг для каждого пользователя из тестовой выборки.

#### Данные
Вам предоставлены данные из приложения МТС Библиотека по взаимодействиям пользователей с книгами за 2 года.

В отдельных файлах есть:

- факты чтения книг пользователями
- описание книг
- описание пользователей

###### Описание данных
В представленном датасете собраны данные по пользователям и книгам, а также по их взаимодействиям (прочтение книги пользователем) из сервиса МТС Библиотека. Данные по чтению пользователями книг собраны за 2 два года, с 01-01-2018 по 31-12-2019 включительно, и разбавлены случайным шумом. ID пользователей и книг анонимизированы.

users.csv
В данном файле содержится информация о пользователях:

- user_id - ID пользователя, int64
- age - возрастная группа пользователя, строка вида "M_N"
 - 18_24 - от 18 до 24 лет включительно
 - 25_34 - от 25 до 34 лет включительно
 - 35_44 - от 35 до 44 лет включительно
 - 45_54 - от 45 до 54 лет включительно
 - 55_64 - от 55 до 64 лет включительно
 - 65_inf - от 65 и старше
 - NaN - неизвестно
- sex - пол пользователя, 1/0
 - 1 - мужчина
 - 0 - женщина
 - NaN - неизвестно
 
items.csv
В данном файле содержится информация о книгах:

- item_id - ID книги, int64
- title - название книги, строка
- genres - жанры, строка с разделителем ','
- authors - авторы, строка с разделителем ','
- year - год публикации, строка -- строка, потому что есть значения, которые нельзя автоматически привести к числовому значению 

interactions.csv
В данном файле содержится информация о взаимодействиях пользователей и книг:

- user_id - ID пользователя, int64
- item_id - ID книги, int64
- progress - прогресс по чтению в процентах, int8
- rating - рейнтинг книги выставленный пользователем, от 1 до 5, много пропущенных значений
- start_date - дата начала чтения книги пользователем

In [2]:
import numpy as np 
import pandas as pd 
import scipy.sparse as sp
from itertools import islice, cycle
from more_itertools import pairwise
from tqdm.auto import tqdm

# Preprocessing

In [3]:
df = pd.read_csv('./interactions.csv')
df_users = pd.read_csv('./users.csv')
df_items = pd.read_csv('./items.csv')

### Interactions

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562617 entries, 0 to 1562616
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   user_id     1562617 non-null  int64  
 1   item_id     1562617 non-null  int64  
 2   progress    1562617 non-null  int64  
 3   rating      323571 non-null   float64
 4   start_date  1562617 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 59.6+ MB


In [4]:
df['start_date'] = pd.to_datetime(df['start_date'])

In [5]:
duplicates = df.duplicated(subset=['user_id', 'item_id'], keep=False) # подсвечивает строки дубликаты как True
df_duplicates = df[duplicates].sort_values(by=['user_id', 'start_date']) # отдельно выделим дуюликаты (когда пользователь несколько раз принимался за одну книгу)
df = df[~duplicates] # остальные данные

In [6]:
df_duplicates = df_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max', # наибольший прогресс прочтения
    'rating': 'max', # наивысший поставленный рейтинг книге
    'start_date': 'min' # когда начал читать впервые
})
df = df.append(df_duplicates.reset_index(), ignore_index=True) # добавили обработанные дубли

In [7]:
df['progress'] = df['progress'].astype(np.int8)
df['rating'] = df['rating'].astype(pd.SparseDtype(np.float32, np.nan))

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562505 entries, 0 to 1562504
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype               
---  ------      --------------    -----               
 0   user_id     1562505 non-null  int64               
 1   item_id     1562505 non-null  int64               
 2   progress    1562505 non-null  int8                
 3   rating      323563 non-null   Sparse[float32, nan]
 4   start_date  1562505 non-null  datetime64[ns]      
dtypes: Sparse[float32, nan](1), datetime64[ns](1), int64(2), int8(1)
memory usage: 39.7 MB


In [9]:
df.to_pickle('interactions_preprocessed.pickle')

### Users

In [10]:
df_users.head()

,user_id,age,sex
0,0,45_54,1.0
1,1,25_34,0.0
2,2,45_54,0.0
3,3,65_inf,0.0
4,4,18_24,0.0


In [11]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  137254 non-null  int64  
 1   age      137244 non-null  object 
 2   sex      135640 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.1+ MB


In [12]:
df_users.nunique()

user_id    137254
age             6
sex             2
dtype: int64

In [13]:
df_users['age'] = df_users['age'].astype('category')
df_users['sex'] = df_users['sex'].astype(pd.SparseDtype(np.float32, np.nan))

In [14]:
interaction_users = df['user_id'].unique() # всего разных пользователей во взаимодействиях

# Возвращает отсортированные уникальные значения, которые есть в обоих входных массивах. 
common_users = len(np.intersect1d(interaction_users, df_users['user_id'])) # и во вз-виях и в списке инфы о юзерах
# Возвращает уникальные значения в ar1, которых нет в ar2. 
users_only_in_interaction = len(np.setdiff1d(interaction_users, df_users['user_id'])) # есть только в вз-виях, но про них нет фичей 
users_only_features = len(np.setdiff1d(df_users['user_id'], interaction_users)) # есть про них фичи, но они не делали ничего с книгами
total_users = common_users + users_only_in_interaction + users_only_features
print(f'Кол-во пользователей - {total_users}')
print(f'Кол-во пользователей c взаимодействиями и фичами - {common_users} ({common_users / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c взаимодействиями - {users_only_in_interaction} ({users_only_in_interaction / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c фичами - {users_only_features} ({users_only_features / total_users * 100:.2f}%)')

Кол-во пользователей - 164771
Кол-во пользователей c взаимодействиями и фичами - 130808 (79.39%)
Кол-во пользователей только c взаимодействиями - 27517 (16.70%)
Кол-во пользователей только c фичами - 6446 (3.91%)


In [15]:
df_users.to_pickle('users_preprocessed.pickle')

### Items

In [16]:
df_items.head()

,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,281311,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,213473,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [17]:
df_items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       63758 non-null  int64 
 1   title    63758 non-null  object
 2   genres   63753 non-null  object
 3   authors  56700 non-null  object
 4   year     49508 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


In [18]:
def num_bytes_format(num_bytes, float_prec=4):
    units = ['bytes', 'Kb', 'Mb', 'Gb', 'Tb', 'Pb', 'Eb']
    for unit in units[:-1]:
        if abs(num_bytes) < 1000:
            return f'{num_bytes:.{float_prec}f} {unit}'
        num_bytes /= 1000
    return f'{num_bytes:.4f} {units[-1]}'

In [19]:
num_bytes = df_items.memory_usage(deep=True).sum()
num_bytes_format(num_bytes)

'30.0965 Mb'

In [20]:
df_items.nunique()

id         63758
title      58093
genres     11091
authors    17024
year        1074
dtype: int64

In [21]:
for col in ['genres', 'authors', 'year']:
    df_items[col] = df_items[col].astype('category')

In [22]:
df_items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   id       63758 non-null  int64   
 1   title    63758 non-null  object  
 2   genres   63753 non-null  category
 3   authors  56700 non-null  category
 4   year     49508 non-null  category
dtypes: category(3), int64(1), object(1)
memory usage: 2.5+ MB


In [23]:
num_bytes = df_items.memory_usage(deep=True).sum()
num_bytes_format(num_bytes) # стало весить меньше после преобразования

'18.6994 Mb'

In [24]:
interaction_items = df['item_id'].unique()

common_items = len(np.intersect1d(interaction_items, df_items['id']))
items_only_in_interaction = len(np.setdiff1d(interaction_items, df_items['id']))
items_only_features = len(np.setdiff1d(df_items['id'], interaction_items))
total_items = common_items + items_only_in_interaction + items_only_features
print(f'Кол-во книг - {total_items}')
print(f'Кол-во книг c взаимодействиями и фичами - {common_items} ({common_items / total_items * 100:.2f}%)')
print(f'Кол-во книг только c взаимодействиями - {items_only_in_interaction} ({items_only_in_interaction / total_items * 100:.2f}%)')
print(f'Кол-во книг только c фичами - {items_only_features} ({items_only_features / total_items * 100:.2f}%)')

Кол-во книг - 63758
Кол-во книг c взаимодействиями и фичами - 63758 (100.00%)
Кол-во книг только c взаимодействиями - 0 (0.00%)
Кол-во книг только c фичами - 0 (0.00%)


In [25]:
df_items.to_pickle('items_preprocessed.pickle')

### Пример поиграть

In [26]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
    'value':   [4,                    5,                    3,            5]
})
df_true

,user_id,item_id,value
0,Аня,Мастер и Маргарита,4
1,Боря,451° по Фаренгейту,5
2,Вася,Зеленая миля,3
3,Вася,Рита Хейуорт и спасение из Шоушенка,5


In [27]:
df_recs = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
    'value':   [3.28,                 3.5,                  4.06,           4.73]
})
df_recs

,user_id,item_id,value
0,Аня,Мастер и Маргарита,3.28
1,Боря,451° по Фаренгейту,3.50
2,Вася,Зеленая миля,4.06
3,Вася,Рита Хейуорт и спасение из Шоушенка,4.73


In [28]:
df_true = df_true.set_index(['user_id', 'item_id']) # указанные колонки стали индексами
df_recs = df_recs.set_index(['user_id', 'item_id'])

df_merged = df_true.join(df_recs, how='left', lsuffix='_true', rsuffix='_recs')
df_merged

value_true  value_recs
user_id item_id                                                    
Аня     Мастер и Маргарита                            4        3.28
Боря    451° по Фаренгейту                            5        3.50
Вася    Зеленая миля                                  3        4.06
        Рита Хейуорт и спасение из Шоушенка           5        4.73

In [29]:
df_merged['MAE'] = (df_merged['value_true'] - df_merged['value_recs']).abs()
df_merged['MSE'] = (df_merged['value_true'] - df_merged['value_recs']) ** 2
df_merged

value_true  value_recs   MAE  \
user_id item_id                                                             
Аня     Мастер и Маргарита                            4        3.28  0.72   
Боря    451° по Фаренгейту                            5        3.50  1.50   
Вася    Зеленая миля                                  3        4.06  1.06   
        Рита Хейуорт и спасение из Шоушенка           5        4.73  0.27   

                                                MSE  
user_id item_id                                      
Аня     Мастер и Маргарита                   0.5184  
Боря    451° по Фаренгейту                   2.2500  
Вася    Зеленая миля                         1.1236  
        Рита Хейуорт и спасение из Шоушенка  0.0729

In [30]:
print(f"MAE  - {df_merged['MAE'].mean():.4f}")
print(f"MSE  - {df_merged['MSE'].mean():.4f}")
print(f"RMSE - {np.sqrt(df_merged['MSE'].mean()):.4f}")

MAE  - 0.8875
MSE  - 0.9912
RMSE - 0.9956


- True positive (TP) - модель рекомендовала объект, с которым пользователь провзаимодействовал
- False positive (FP) - модель рекомендовала объект, с которым пользователь не провзаимодействовал
- True negative (TN) - модель не рекомендовала объект, с которым пользователь не провзаимодействовал
- False negative (FN) - модель не рекомендовала объект, с которым пользователь провзаимодействовал

In [31]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})
df_true

,user_id,item_id
0,Аня,Мастер и Маргарита
1,Боря,451° по Фаренгейту
2,Вася,Зеленая миля
3,Вася,Рита Хейуорт и спасение из Шоушенка


In [32]:
df_recs = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня', 
        'Боря', 'Боря', 'Боря', 
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени', 
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Искра жизни', 'Зеленая миля', 
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})
df_recs

,user_id,item_id,rank
0,Аня,Отверженные,1
1,Аня,Двенадцать стульев,2
2,Аня,Герои нашего времени,3
3,Боря,451° по Фаренгейту,1
4,Боря,1984,2
5,Боря,О дивный новый мир,3
6,Вася,Десять негритят,1
7,Вася,Искра жизни,2
8,Вася,Зеленая миля,3


In [33]:
df_merged = df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Зеленая миля                          3.0
        Рита Хейуорт и спасение из Шоушенка   NaN

Вначале посчитаем метрик для топ-2 (т.е. К = 2). Алгоритм следующий:

Релевантные объекты, которые не были рекомендованы игнорируем (NaN)  
Определяем, какие релеватные рекомендации попали в топ-2 (hit)  
True positive для каждого пользователя
Делим TP на K   
Считаем Precision@K для каждого пользователя как сумму его TP/K  
Все Precision@K усредняем

In [34]:
k = 2

In [35]:
df_merged['hit@2'] = df_merged['rank'] <= k
df_merged

rank  hit@2
user_id item_id                                         
Аня     Мастер и Маргарита                    NaN  False
Боря    451° по Фаренгейту                    1.0   True
Вася    Зеленая миля                          3.0  False
        Рита Хейуорт и спасение из Шоушенка   NaN  False

In [36]:
df_merged['hit@2/2'] = df_merged['hit@2'] / k
df_merged

rank  hit@2  hit@2/2
user_id item_id                                                  
Аня     Мастер и Маргарита                    NaN  False      0.0
Боря    451° по Фаренгейту                    1.0   True      0.5
Вася    Зеленая миля                          3.0  False      0.0
        Рита Хейуорт и спасение из Шоушенка   NaN  False      0.0

In [37]:
df_prec2 = df_merged.groupby(level=0)['hit@2/2'].sum()
df_prec2

user_id
Аня     0.0
Боря    0.5
Вася    0.0
Name: hit@2/2, dtype: float64

In [38]:
print(f'Precision@2 - {df_prec2.mean()}')

Precision@2 - 0.16666666666666666


In [39]:
# или так
df_merged['hit@2/2'].sum() / df_merged.index.get_level_values('user_id').nunique()

0.16666666666666666

In [40]:
users_count = df_merged.index.get_level_values('user_id').nunique()
for k in [1, 2, 3]:
    hit_k = f'hit@{k}'
    df_merged[hit_k] = df_merged['rank'] <= k
    print(f'Precision@{k} = {(df_merged[hit_k] / k).sum() / users_count:.4f}')

Precision@1 = 0.3333
Precision@2 = 0.1667
Precision@3 = 0.2222


In [41]:
df_merged['users_item_count'] = df_merged.groupby(level='user_id')['rank'].transform(np.size)
df_merged

rank  hit@2  hit@2/2  hit@1  \
user_id item_id                                                            
Аня     Мастер и Маргарита                    NaN  False      0.0  False   
Боря    451° по Фаренгейту                    1.0   True      0.5   True   
Вася    Зеленая миля                          3.0  False      0.0  False   
        Рита Хейуорт и спасение из Шоушенка   NaN  False      0.0  False   

                                             hit@3  users_item_count  
user_id item_id                                                       
Аня     Мастер и Маргарита                   False               1.0  
Боря    451° по Фаренгейту                    True               1.0  
Вася    Зеленая миля                          True               2.0  
        Рита Хейуорт и спасение из Шоушенка  False               2.0

In [42]:
for k in [1, 2, 3]:
    hit_k = f'hit@{k}'
    # Уже посчитано
    # df_merged[hit_k] = df_merged['rank'] <= k  
    print(f"Recall@{k} = {(df_merged[hit_k] / df_merged['users_item_count']).sum() / users_count:.4f}")

Recall@1 = 0.3333
Recall@2 = 0.3333
Recall@3 = 0.5000


### Ranking
Эти метрики оценивают качество топ-N рекомендаций c учетом рангов/позиций. Основная идея - оценить "попадания" с весом, зависящим от позиции (обычно это обратная пропорциальная зависимость, то есть чем больше позиция, тем меньше вес).

In [43]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})
df_true 

,user_id,item_id
0,Аня,Мастер и Маргарита
1,Боря,451° по Фаренгейту
2,Вася,Зеленая миля
3,Вася,Рита Хейуорт и спасение из Шоушенка


In [44]:
df_recs = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня', 
        'Боря', 'Боря', 'Боря', 
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени', 
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Рита Хейуорт и спасение из Шоушенка', 'Зеленая миля', 
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})
df_recs

,user_id,item_id,rank
0,Аня,Отверженные,1
1,Аня,Двенадцать стульев,2
2,Аня,Герои нашего времени,3
3,Боря,451° по Фаренгейту,1
4,Боря,1984,2
5,Боря,О дивный новый мир,3
6,Вася,Десять негритят,1
7,Вася,Рита Хейуорт и спасение из Шоушенка,2
8,Вася,Зеленая миля,3


In [45]:
df_merged = df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')
df_merged = df_merged.sort_values(by=['user_id', 'rank'])
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Рита Хейуорт и спасение из Шоушенка   2.0
        Зеленая миля                          3.0

In [46]:
df_merged['reciprocal_rank'] = 1 / df_merged['rank']
df_merged

rank  reciprocal_rank
user_id item_id                                                   
Аня     Мастер и Маргарита                    NaN              NaN
Боря    451° по Фаренгейту                    1.0         1.000000
Вася    Рита Хейуорт и спасение из Шоушенка   2.0         0.500000
        Зеленая миля                          3.0         0.333333

In [47]:
mrr = df_merged.groupby(level='user_id')['reciprocal_rank'].max()
mrr

user_id
Аня     NaN
Боря    1.0
Вася    0.5
Name: reciprocal_rank, dtype: float64

In [48]:
print(f"MRR = {mrr.fillna(0).mean()}")

MRR = 0.5


In [49]:
df_merged['cumulative_rank'] = df_merged.groupby(level='user_id').cumcount() + 1
df_merged['cumulative_rank'] = df_merged['cumulative_rank'] / df_merged['rank']
df_merged['users_item_count'] = df_merged.groupby(level='user_id')['rank'].transform(np.size)
df_merged

rank  reciprocal_rank  \
user_id item_id                                                      
Аня     Мастер и Маргарита                    NaN              NaN   
Боря    451° по Фаренгейту                    1.0         1.000000   
Вася    Рита Хейуорт и спасение из Шоушенка   2.0         0.500000   
        Зеленая миля                          3.0         0.333333   

                                             cumulative_rank  users_item_count  
user_id item_id                                                                 
Аня     Мастер и Маргарита                               NaN               1.0  
Боря    451° по Фаренгейту                          1.000000               1.0  
Вася    Рита Хейуорт и спасение из Шоушенка         0.500000               2.0  
        Зеленая миля                                0.666667               2.0

In [50]:
users_count = df_merged.index.get_level_values('user_id').nunique()
map3 = (df_merged["cumulative_rank"] / df_merged["users_item_count"]).sum() / users_count
print(f"MAP@3 = {map3}")

MAP@3 = 0.5277777777777778
